In [11]:
#Create user, activity and traclpoint documents with some dummy data: 

from DbConnector import DbConnector
from example import *
from sqlite3 import Cursor

In [61]:
def task_1(connection: DbConnector): 
    users: Cursor = connection.db["user"].find({})
    activities: Cursor = connection.db["activities"].find({})
    trackpoints: Cursor = connection.db["trackpoint"].find({})
    print(f'Number of users: {len(list(users))}')
    print(f'Number of activities: {len(list(activities))}')
    print(f'Number of trackpoints: {len(list(trackpoints))}')
    
    

In [ ]:
def task_2(connection: DbConnector): 
    users: Cursor = connection.db["user"].find({})
    
    

In [62]:
def main(): 
    conn: DbConnector = DbConnector()
    task_1(conn)
    conn.close_connection()

main()


You are connected to the database: my_db
-----------------------------------------------

Number of users: 2
Number of activities: 2
Number of trackpoints: 2

-----------------------------------------------
Connection to my_db-db is closed
